In [1]:
from flowstab.flow_stability import FlowStability
from flowstab.network_clustering import Clustering,avg_norm_var_information, run_multi_louvain
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.read_csv("Pre_processed_Data/primary_school_day2.csv")
real_taus = np.logspace(-2, 7, 100)

for index in range(10):
    taus=[i for x, i in enumerate(real_taus) if x%10==index]
    fs = FlowStability()
    fs.set_temporal_network(events_table=df, relabel_nodes=True)
    fs.compute_laplacian_matrices()
    fs.set_time_scale(taus)
    fs.compute_inter_transition_matrices()
    fs.time_direction = 0          
    fs.set_flow_clustering()
    fs._temporal_network.inter_T=None


    n_repetitions=200

    for tau in tqdm(taus):
        forward_partitions = {}
        clus_obj_for=fs.flow_clustering_forward[tau]
        _, cluster_lists, stabilities, _=run_multi_louvain(Clustering(p1=clus_obj_for.p1, p2=None,S=clus_obj_for.I_list[0]), num_repeat=n_repetitions)
        forward_partitions['clusters']=cluster_lists
        forward_partitions['stabilities']=stabilities
        np.save(f"Forward_partitions/forward_partitions-tau={tau}.npy", forward_partitions, allow_pickle=True)
        fs.flow_clustering_forward[tau]=None


    for tau in tqdm(taus):
        backward_partitions = {}
        clus_obj_back=fs.flow_clustering_backward[tau]
        n_loops, cluster_lists, stabilities, seeds=run_multi_louvain(Clustering(p1=clus_obj_back.p1, p2=None,S=clus_obj_back.I_list[0]), num_repeat=n_repetitions)
        backward_partitions['clusters']=cluster_lists
        backward_partitions['stabilities']=stabilities
        np.save(f"Backward_partitions/backward_partitions-tau={tau}.npy", backward_partitions, allow_pickle=True)
        fs.flow_clustering_backward[tau]=None

    fs=None

Could not load sparse_dot_mkl. Will use scipy.sparse for matrix products.


2025-11-12 01:35:07,537 - INFO - flowstab\flow_stability.py:235 - PID:11332 - Set an empty temporal network.
2025-11-12 01:35:07,547 - INFO - flowstab\flow_stability.py:139 - PID:11332 - Successfully initiated a FlowStability instance!
2025-11-12 01:35:07,555 - INFO - flowstab\flow_stability.py:230 - PID:11332 - Set the temporal network: <class 'flowstab.temporal_network.ContTempNetwork'> with 238 nodes and 65150 events
2025-11-12 01:35:07,556 - INFO - flowstab\temporal_network.py:1176 - PID:11332 - Computing Laplacians
2025-11-12 01:35:07,686 - INFO - flowstab\temporal_network.py:1266 - PID:11332 - 0 over 1545
2025-11-12 01:35:07,687 - INFO - flowstab\temporal_network.py:1270 - PID:11332 - 0.02s
2025-11-12 01:35:38,620 - INFO - flowstab\temporal_network.py:1266 - PID:11332 - 1000 over 1545
2025-11-12 01:35:38,620 - INFO - flowstab\temporal_network.py:1270 - PID:11332 - 30.95s
2025-11-12 01:35:49,975 - INFO - flowstab\temporal_network.py:1326 - PID:11332 - Finished in 42.30512642860412